# InceptionNet
the concept behind inception net to reduce the depth of neural network and increase widht of network also we can say parallel filter implementation. <br>
The elemnent contain this network are given below

<li> **Inception Module:** that contain parallel computation on input.<br>
<li> **Axulariy Layer:** that in middle of network that help's to avoid "Gradient Vanshing" problem.<br>
<li> Main Module that start's with simple pattern like <br> ZeroPadding2d-->Conv2D-->MaxPooling2D-->Inception_blocks--> Flatten-->Dense-->Softmax  

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
%matplotlib inline

## Inception block
Inception Block where different type of filter apply on intial input or previous input parallelly.<br>
Types of filter:<br>
<li> 1_x_1
<li> 3_x_3
<li> 5_x_5
<li> MaxPooling2D

In [ ]:
def inception_block(X, 1_x_1, 3_x_3_filter_reduce, 3_x_3_filter, 5_x_5_filter_reduce, 5_x_5_filter, p_pool):
  X_1 = Conv2D(1_x_1, (1, 1), padding = 'same', activation = 'relu')(X)

  X_2 = Conv2D(3_x_3_filter_reduce, (1, 1), padding = 'same', activation = 'relu')(X) # reduction layer
  X_2 = Conv2D(3_x_3_filter, (3, 3), padding = 'same', activation = 'relu')(X_2) # apply filter of 3x3 and 3_x_3_filters

  X_3 = Conv2D(5_x_5_filter_reduce, (1, 1), padding = 'same', activation = 'relu')(X) # reduction of layers 1_x_1 convolve layer of 5_x_5_filter reduction
  X_3 = Conv2D(5_x_5_filter, (5, 5), padding = 'same', activation = 'relu')(X_3) # Apply kernel size 5x5 and filter 5_x_5_filter

  x_4 = MaxPooling2D((3, 3), strides = (1, 1), padding = 'same')(X) # Apply Maxpooling on initial layer
  x_4 = Conv2D(p_pool, (1, 1), padding = 'same', activation = 'relu')(x_4)

  X = layers.concatenate([X_1, X_2, X_3, x_4], axis = -1) # Apply concatenation function of all layers in output like its stake to gether just like connect one by one
  return X

## AXULARIY Block
Axulariy Block work like end layer connected at the end model fully connected layer but it present in middle.

In [ ]:
def Axulariy_layer(X):
  X = AveragePooling2D((5, 5), strides = (3, 3), padding = 'valid')(X)
  X = Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(X)
  X = Flatten()(X)
  X = Dense(1024, activation = 'relu')(X)
  X = Dropout(0.7)(X)
  X = Dense(6, activation = 'softmax')(X)
  return X

In [ ]:
def inception(input_shape = (228, 228, 3), class = 6):
  X_input = Input(input_shape)
  X = ZeroPadding2D((3, 3))(X_input)

  X = Conv2D(64, (7, 7), strides = (2, 2), padding = 'valid', activation = 'relu')(X)
  X = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(X)

  X = Conv2D(64, (1, 1), strides = (1, 1), padding = 'same', activation = 'relu')(X)
  X = Conv2D(192, (3, 3), strides = (1, 1), padding = 'same', activation = 'relu')(X)

  #Inception_block a1, a2
  X = inception_block(X, 64, 96, 128, 16, 32,32)
  X = inception_block (X, 128, 128, 192, 32, 96, 64)

  #MaxPooling
  X = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(X)

  #inception block a4, a5, a6, a7, a8
  X = inception_block(X, 192, 96, 208, 16, 48, 64)

  #Axulariy Block To Avoid "Gradient Vanshing Problem"
  X_Aux = X
  x_Aux = Axulariy_layer(X_Aux)
  X = inception_block(X, 160, 112, 224, 24, 64, 64)
  X = inception_block(X, 128, 128, 256, 24, 64, 64)
  X = inception_block(X, 112, 144, 288, 32, 64, 64)

  #Axulariy BLOCK
  X_Aux = X
  x_Aux = Axulariy_layer(X_Aux)

  X = inception_block(X, 256, 160, 320, 32, 128, 128)

  X = MaxPooling2D((3, 3), strides = (2, 2), padding = 'same')(X)

  #inception_block a9, a10
  X = inception_block(X, 256, 160, 320, 32, 128, 128)
  X = inception_block(X, 384, 192, 384, 48, 128, 128)

  # AveragePooling2D
  X = AveragePooling2D((7, 7), padding = 'same')(X)

  # Dimenstion change 2D to 1D for FC layer input type
  X = Flatten()(X)

  # Fully Connected Layer
  X = Dense(1000, activation = 'softmax')(X)

  model = Model(inputs = X_input, outputs = X)

In [ ]:
model = inception(x_input, class = 10)
Print(model.summary)